In [2]:
import pandas as pd
import numpy as np
import megamerge

In [1]:
segments = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to"],
    data=[
        ["H001", "L",   0, 100],
        ["H001", "L", 100, 200],
        ["H001", "L", 200, 300],
        ["H001", "L", 300, 400],

        ["H001", "R",   0, 100],
    ]
)

data = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to", "measure", "category"],
    data=[                                   # overlaps lengths
        ["H001", "L", 50,  140,  1.0, "A"],  # 50  40   0  0
        ["H001", "L", 140, 160,  2.0, "B"],  #  0  20   0  0
        ["H001", "L", 160, 180,  3.0, "B"],  #  0  20   0  0
        ["H001", "L", 180, 220,  4.0, "B"],  #  0  20  20  0
        ["H001", "L", 220, 240,  5.0, "C"],  #  0   0  20  0
        ["H001", "L", 240, 260,  5.0, "C"],  #  0   0  20  0
        ["H001", "L", 260, 280,  6.0, "D"],  #  0   0  20  0
        ["H001", "L", 280, 300,  7.0, "E"],  #  0   0  20  0
        ["H001", "L", 300, 320,  8.0, "F"],  #  0   0     20

        ["H001", "R",  10,  80,  9.0, "G"],  #  0   0     20
        ["H001", "R",  80, 120, 10.0, "H"],  #  0   0     20
    ]
)

expected_result = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to",  "measure longest value",  "category longest value"],
    data=[
        ["H001", "L",   0, 100,  1.0,  "A"],
        ["H001", "L", 100, 200,  1.0,  "B"],
        ["H001", "L", 200, 300,  5.0,  "C"],
        ["H001", "L", 300, 400,  8.0,  "F"],

        ["H001", "R",   0, 100,  9.0,  "G"],
    ]
)

NameError: name 'pd' is not defined

In [2]:
res = megamerge.merge_interval_index(
    segments.set_index(["road","cwy"]).loc[("H001", "L"),["slk_from", "slk_to"]].to_numpy(dtype="f8"),
    data    .set_index(["road","cwy"]).loc[("H001", "L"),["slk_from", "slk_to"]].to_numpy(dtype="f8"),
    0
)

NameError: name 'megamerge' is not defined

In [5]:
list(res)

[(array([0], dtype=uint64), array([50.])),
 (array([0, 1, 2, 3], dtype=uint64), array([40., 20., 20., 20.])),
 (array([3, 4, 5, 6, 7], dtype=uint64), array([20., 20., 20., 20., 20.])),
 (array([8], dtype=uint64), array([20.]))]

In [8]:
# generate segmentation
big_segments = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to"],
    data   =[
        ["H001", "L",   index*500, (index+1)*500] for index in range(int(3000 / (3000/60_000*10)))
    ]
)

big_data = pd.DataFrame(
    columns= ["road", "cwy", "slk_from", "slk_to", "measure"],
    data   = [
        ["H001", "L",   (index)*10, (index+1)*10, index] for index in range(int(3000 / 0.01))
    ]
)
print(f"{len(big_segments) = :,.0f} and {len(big_data) = :,.0f}")

len(big_segments) = 6,000 and len(big_data) = 300,000


In [9]:
res = megamerge.merge_interval_index(
    big_segments.set_index(["road","cwy"]).loc[("H001", "L"),["slk_from", "slk_to"]].to_numpy(dtype="f8"),
    big_data    .set_index(["road","cwy"]).loc[("H001", "L"),["slk_from", "slk_to"]].to_numpy(dtype="f8"),
    0
)
res_list = list(res)

In [10]:
result = []
for index, overlap in res_list:
    data = big_data["measure"].iloc[index]
    namask = ~data.isna()
    data = data[namask]
    overlap = overlap[namask]
    
    result.append(
        np.sum(data*overlap/np.sum(overlap))
    )
result

[24.5,
 74.5,
 124.5,
 174.5,
 224.5,
 274.5,
 324.49999999999994,
 374.49999999999994,
 424.50000000000006,
 474.50000000000006,
 524.5000000000001,
 574.5,
 624.5,
 674.5,
 724.5,
 774.5,
 824.5,
 874.5,
 924.5,
 974.5,
 1024.5,
 1074.5,
 1124.5,
 1174.5,
 1224.5,
 1274.5,
 1324.5,
 1374.5,
 1424.5,
 1474.5,
 1524.5,
 1574.5,
 1624.5,
 1674.5,
 1724.5,
 1774.5,
 1824.5,
 1874.5,
 1924.5,
 1974.5,
 2024.5,
 2074.5,
 2124.5,
 2174.5000000000005,
 2224.5000000000005,
 2274.5000000000005,
 2324.5000000000005,
 2374.5000000000005,
 2424.5000000000005,
 2474.5000000000005,
 2524.5000000000005,
 2574.5000000000005,
 2624.5000000000005,
 2674.5000000000005,
 2724.5000000000005,
 2774.5000000000005,
 2824.5000000000005,
 2874.5000000000005,
 2924.5000000000005,
 2974.5000000000005,
 3024.5000000000005,
 3074.5000000000005,
 3124.5000000000005,
 3174.5000000000005,
 3224.4999999999995,
 3274.4999999999995,
 3324.4999999999995,
 3374.4999999999995,
 3424.4999999999995,
 3474.4999999999995,
 352

In [33]:
len(big_data)

300000

In [34]:
len(big_segments)

300000

300_000*300_000 takes 6 minutes to merge 1 column (plus a tiny bit extra per additional column). This is the absolute outside worst case for a single interval merge.  This is a 10th of the road network at 10 metre segmentation. Since all other segments are far shorter, we can expect that the total time will be significantly less than the upper bound of 10 * 6 minutes